# Imports and configs

In [89]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif, f_classif
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
import optuna
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [3]:
data = pd.read_csv(r'modelling_data\concatenated_data\cleaned_data\all_games_1819_2324_cleaned.csv')

In [4]:
data.shape

(37339, 162)

In [5]:
data.head()

,attendance,home_team_name,away_team_name,game_week,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,home_team_xg,away_team_xg,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_ht_fhg_percentage_pre_match,over_05_ht_fhg_percentage_pre_match,over_15_2hg_percentage_pre_match,over_05_2hg_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,home_team_cumulative_goals_scored_pre_game,away_team_cumulative_goals_scored_pre_game,home_team_cumulative_goals_conceded_pre_game,away_team_cumulative_goals_conceded_pre_game,home_team_average_goals_scored_per_game_pre_game,away_team_average_goals_scored_per_game_pre_game,home_team_average_goals_conceded_per_game_pre_game,away_team_average_goals_conceded_per_game_pre_game,home_team_average_first_half_goals_scored_pre_game,home_team_average_second_half_goals_scored_pre_game,away_team_average_first_half_goals_scored_pre_game,away_team_average_second_half_goals_scored_pre_game,home_team_average_first_half_goals_conceded_pre_game,home_team_average_second_half_goals_conceded_pre_game,away_team_average_first_half_goals_conceded_pre_game,away_team_average_second_half_goals_conceded_pre_game,home_team_average_first_half_goals_total_pre_game,home_team_average_second_half_goals_total_pre_game,away_team_average_first_half_goals_total_pre_game,away_team_average_second_half_goals_total_pre_game,home_team_average_corners_total_per_game_pre_game,away_team_average_corners_total_per_game_pre_game,home_team_average_corners_per_game_pre_game,away_team_average_corners_per_game_pre_game,home_team_average_yellow_cards_total_per_game_pre_game,away_team_average_yellow_cards_total_per_game_pre_game,home_team_average_yellow_cards_per_game_pre_game,away_team_average_yellow_cards_per_game_pre_game,home_team_cumulative_red_cards_pre_game,away_team_cumulative_red_cards_pre_game,home_team_average_red_cards_total_per_game_pre_game,away_team_average_red_cards_total_per_game_pre_game,home_team_average_red_cards_per_game_pre_game,away_team_average_red_cards_per_game_pre_game,home_team_average_shots_per_game_pre_game,home_team_average_shots_on_target_per_game_pre_game,away_team_average_shots_per_game_pre_game,away_team_average_shots_on_target_per_game_pre_game,home_team_average_fouls_per_game_pre_game,away_team_average_fouls_per_game_pre_game,home_team_average_fouls_total_per_game_pre_game,away_team_average_fouls_total_per_game_pre_game,home_team_average_possession_per_game_pre_game,away_team_average_possession_per_game_pre_game,home_team_average_xg_per_game_pre_game,away_team_average_xg_per_game_pre_game,home_team_games_without_goals_pre_game,away_team_games_without_goals_pre_game,home_team_cumulative_btts_pre_game,away_team_cumulative_btts_pre_game,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_te

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37339 entries, 0 to 37338
Columns: 162 entries, attendance to average_corners_h2h
dtypes: float64(105), int64(51), object(6)
memory usage: 46.1+ MB


In [8]:
data.groupby(['league', 'season']).apply(lambda x: x['game_week'].isna().sum())

league              season
austria             18/19     0
                    19/20     0
                    20/21     0
                    21/22     0
                    22/23     0
                    23/24     0
belgium pro league  18/19     0
                    19/20     0
                    20/21     0
                    21/22     0
                    22/23     0
                    23/24     0
bundesliga          18/19     0
                    19/20     0
                    20/21     0
                    21/22     0
                    22/23     0
                    23/24     0
czech first league  18/19     0
                    19/20     0
                    20/21     0
                    21/22     0
                    22/23     0
                    23/24     0
denmark             18/19     0
                    19/20     0
                    20/21     0
                    21/22     0
                    22/23     0
                    23/24     0
ekstraklasa  

In [9]:
data.dtypes

attendance                                                  float64
home_team_name                                               object
away_team_name                                               object
game_week                                                   float64
home_team_goal_count                                          int64
away_team_goal_count                                          int64
total_goal_count                                              int64
total_goals_at_half_time                                      int64
home_team_goal_count_half_time                                int64
away_team_goal_count_half_time                                int64
home_team_corner_count                                        int64
away_team_corner_count                                        int64
home_team_yellow_cards                                        int64
home_team_red_cards                                           int64
away_team_yellow_cards                          

In [10]:
data['result'] = data['result'].astype('category')

In [11]:
def scale_data_arcsinh_std(df):
    num_features = df.select_dtypes(include='number').columns
    df[num_features] = np.arcsinh(df[num_features])
    df[num_features] = StandardScaler().fit_transform(df[num_features])
    return df

In [12]:
data = scale_data_arcsinh_std(data)

In [13]:
data.tail()

,attendance,home_team_name,away_team_name,game_week,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,home_team_xg,away_team_xg,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_ht_fhg_percentage_pre_match,over_05_ht_fhg_percentage_pre_match,over_15_2hg_percentage_pre_match,over_05_2hg_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,home_team_cumulative_goals_scored_pre_game,away_team_cumulative_goals_scored_pre_game,home_team_cumulative_goals_conceded_pre_game,away_team_cumulative_goals_conceded_pre_game,home_team_average_goals_scored_per_game_pre_game,away_team_average_goals_scored_per_game_pre_game,home_team_average_goals_conceded_per_game_pre_game,away_team_average_goals_conceded_per_game_pre_game,home_team_average_first_half_goals_scored_pre_game,home_team_average_second_half_goals_scored_pre_game,away_team_average_first_half_goals_scored_pre_game,away_team_average_second_half_goals_scored_pre_game,home_team_average_first_half_goals_conceded_pre_game,home_team_average_second_half_goals_conceded_pre_game,away_team_average_first_half_goals_conceded_pre_game,away_team_average_second_half_goals_conceded_pre_game,home_team_average_first_half_goals_total_pre_game,home_team_average_second_half_goals_total_pre_game,away_team_average_first_half_goals_total_pre_game,away_team_average_second_half_goals_total_pre_game,home_team_average_corners_total_per_game_pre_game,away_team_average_corners_total_per_game_pre_game,home_team_average_corners_per_game_pre_game,away_team_average_corners_per_game_pre_game,home_team_average_yellow_cards_total_per_game_pre_game,away_team_average_yellow_cards_total_per_game_pre_game,home_team_average_yellow_cards_per_game_pre_game,away_team_average_yellow_cards_per_game_pre_game,home_team_cumulative_red_cards_pre_game,away_team_cumulative_red_cards_pre_game,home_team_average_red_cards_total_per_game_pre_game,away_team_average_red_cards_total_per_game_pre_game,home_team_average_red_cards_per_game_pre_game,away_team_average_red_cards_per_game_pre_game,home_team_average_shots_per_game_pre_game,home_team_average_shots_on_target_per_game_pre_game,away_team_average_shots_per_game_pre_game,away_team_average_shots_on_target_per_game_pre_game,home_team_average_fouls_per_game_pre_game,away_team_average_fouls_per_game_pre_game,home_team_average_fouls_total_per_game_pre_game,away_team_average_fouls_total_per_game_pre_game,home_team_average_possession_per_game_pre_game,away_team_average_possession_per_game_pre_game,home_team_average_xg_per_game_pre_game,away_team_average_xg_per_game_pre_game,home_team_games_without_goals_pre_game,away_team_games_without_goals_pre_game,home_team_cumulative_btts_pre_game,away_team_cumulative_btts_pre_game,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_te

In [15]:
#conversion float64 and int64 to float32 and int32 to optimize computing
data[data.select_dtypes(include='float64').columns] = data[data.select_dtypes(include='float64').columns].astype('float32')
data[data.select_dtypes(include='int64').columns] = data[data.select_dtypes(include='int64').columns].astype('int32')

In [16]:
data.dtypes

attendance                                                   float32
home_team_name                                                object
away_team_name                                                object
game_week                                                    float32
home_team_goal_count                                         float32
away_team_goal_count                                         float32
total_goal_count                                             float32
total_goals_at_half_time                                     float32
home_team_goal_count_half_time                               float32
away_team_goal_count_half_time                               float32
home_team_corner_count                                       float32
away_team_corner_count                                       float32
home_team_yellow_cards                                       float32
home_team_red_cards                                          float32
away_team_yellow_cards            

# Feature selection

In [17]:
#dropping info leaking features
X = data.drop([
    'home_team_name',
    'away_team_name',
    'home_team_goal_count',
    'away_team_goal_count',
    'total_goal_count',
    'total_goals_at_half_time',
    'home_team_goal_count_half_time',
    'away_team_goal_count_half_time',
    'date',
    'time',
    'result',
    'league',
    'season'
], axis=1)

In [18]:
y = data['result'].astype(int)

In [20]:
X

,attendance,game_week,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,home_team_xg,away_team_xg,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_ht_fhg_percentage_pre_match,over_05_ht_fhg_percentage_pre_match,over_15_2hg_percentage_pre_match,over_05_2hg_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,home_team_cumulative_goals_scored_pre_game,away_team_cumulative_goals_scored_pre_game,home_team_cumulative_goals_conceded_pre_game,away_team_cumulative_goals_conceded_pre_game,home_team_average_goals_scored_per_game_pre_game,away_team_average_goals_scored_per_game_pre_game,home_team_average_goals_conceded_per_game_pre_game,away_team_average_goals_conceded_per_game_pre_game,home_team_average_first_half_goals_scored_pre_game,home_team_average_second_half_goals_scored_pre_game,away_team_average_first_half_goals_scored_pre_game,away_team_average_second_half_goals_scored_pre_game,home_team_average_first_half_goals_conceded_pre_game,home_team_average_second_half_goals_conceded_pre_game,away_team_average_first_half_goals_conceded_pre_game,away_team_average_second_half_goals_conceded_pre_game,home_team_average_first_half_goals_total_pre_game,home_team_average_second_half_goals_total_pre_game,away_team_average_first_half_goals_total_pre_game,away_team_average_second_half_goals_total_pre_game,home_team_average_corners_total_per_game_pre_game,away_team_average_corners_total_per_game_pre_game,home_team_average_corners_per_game_pre_game,away_team_average_corners_per_game_pre_game,home_team_average_yellow_cards_total_per_game_pre_game,away_team_average_yellow_cards_total_per_game_pre_game,home_team_average_yellow_cards_per_game_pre_game,away_team_average_yellow_cards_per_game_pre_game,home_team_cumulative_red_cards_pre_game,away_team_cumulative_red_cards_pre_game,home_team_average_red_cards_total_per_game_pre_game,away_team_average_red_cards_total_per_game_pre_game,home_team_average_red_cards_per_game_pre_game,away_team_average_red_cards_per_game_pre_game,home_team_average_shots_per_game_pre_game,home_team_average_shots_on_target_per_game_pre_game,away_team_average_shots_per_game_pre_game,away_team_average_shots_on_target_per_game_pre_game,home_team_average_fouls_per_game_pre_game,away_team_average_fouls_per_game_pre_game,home_team_average_fouls_total_per_game_pre_game,away_team_average_fouls_total_per_game_pre_game,home_team_average_possession_per_game_pre_game,away_team_average_possession_per_game_pre_game,home_team_average_xg_per_game_pre_game,away_team_average_xg_per_game_pre_game,home_team_games_without_goals_pre_game,away_team_games_without_goals_pre_game,home_team_cumulative_btts_pre_game,away_team_cumulative_btts_pre_game,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_average_points_per_game_pre_game,away_team_average_points_per_game_pre_game,home_team_wins_in_last_5_games,home_team_draws_in_last_5_games,home_team_losses_in_last_5_games,away_team_wins_in_last_5_games,a

In [24]:
from sklearn.feature_selection import VarianceThreshold

# Przykład użycia z wartością progową wariancji równą 0.1
selector = VarianceThreshold(threshold=1)
X_new = selector.fit_transform(X)

# Wybrane cechy można sprawdzić tak:
selected_features = X.columns[selector.get_support(indices=True)]
print("Wybrane cechy:", selected_features)

Wybrane cechy: Index(['attendance', 'home_team_corner_count', 'away_team_corner_count',
       'away_team_yellow_cards', 'away_team_red_cards',
       'home_team_second_half_cards', 'away_team_second_half_cards',
       'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'away_team_possession', 'away_team_xg', 'over_25_percentage_pre_match',
       'over_15_ht_fhg_percentage_pre_match',
       'over_05_ht_fhg_percentage_pre_match',
       'over_15_2hg_percentage_pre_match', 'over_05_2hg_percentage_pre_match',
       'average_corners_per_match_pre_match', 'odds_ft_home_team_win',
       'odds_ft_draw', 'odds_ft_away_team_win', 'odds_ft_over35',
       'yellow_cards_total', 'cards_total',
       'away_team_cumulative_goals_conceded_pre_game',
       'home_team_average_goals_scored_per_game_pre_game',
       'away_team_average_first_half_goals_sco

In [19]:
from sklearn.feature_selection import SelectKBest, chi2

# Selekcja zmiennych
selector = SelectKBest(score_func=chi2, k=10)  # Wybierz 10 najlepszych cech
X_new = selector.fit_transform(X, y)

# Wybrane cechy można sprawdzić tak:
selected_features = X.columns[selector.get_support(indices=True)]
print("Wybrane cechy:", selected_features)

ValueError: Input X must be non-negative.

In [154]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
num_features = X.select_dtypes('number').columns

In [32]:
X[num_features]

,attendance,game_week,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,home_team_xg,away_team_xg,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_ht_fhg_percentage_pre_match,over_05_ht_fhg_percentage_pre_match,over_15_2hg_percentage_pre_match,over_05_2hg_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,home_team_cumulative_goals_scored_pre_game,away_team_cumulative_goals_scored_pre_game,home_team_cumulative_goals_conceded_pre_game,away_team_cumulative_goals_conceded_pre_game,home_team_average_goals_scored_per_game_pre_game,away_team_average_goals_scored_per_game_pre_game,home_team_average_goals_conceded_per_game_pre_game,away_team_average_goals_conceded_per_game_pre_game,home_team_average_first_half_goals_scored_pre_game,home_team_average_second_half_goals_scored_pre_game,away_team_average_first_half_goals_scored_pre_game,away_team_average_second_half_goals_scored_pre_game,home_team_average_first_half_goals_conceded_pre_game,home_team_average_second_half_goals_conceded_pre_game,away_team_average_first_half_goals_conceded_pre_game,away_team_average_second_half_goals_conceded_pre_game,home_team_average_first_half_goals_total_pre_game,home_team_average_second_half_goals_total_pre_game,away_team_average_first_half_goals_total_pre_game,away_team_average_second_half_goals_total_pre_game,home_team_average_corners_total_per_game_pre_game,away_team_average_corners_total_per_game_pre_game,home_team_average_corners_per_game_pre_game,away_team_average_corners_per_game_pre_game,home_team_average_yellow_cards_total_per_game_pre_game,away_team_average_yellow_cards_total_per_game_pre_game,home_team_average_yellow_cards_per_game_pre_game,away_team_average_yellow_cards_per_game_pre_game,home_team_cumulative_red_cards_pre_game,away_team_cumulative_red_cards_pre_game,home_team_average_red_cards_total_per_game_pre_game,away_team_average_red_cards_total_per_game_pre_game,home_team_average_red_cards_per_game_pre_game,away_team_average_red_cards_per_game_pre_game,home_team_average_shots_per_game_pre_game,home_team_average_shots_on_target_per_game_pre_game,away_team_average_shots_per_game_pre_game,away_team_average_shots_on_target_per_game_pre_game,home_team_average_fouls_per_game_pre_game,away_team_average_fouls_per_game_pre_game,home_team_average_fouls_total_per_game_pre_game,away_team_average_fouls_total_per_game_pre_game,home_team_average_possession_per_game_pre_game,away_team_average_possession_per_game_pre_game,home_team_average_xg_per_game_pre_game,away_team_average_xg_per_game_pre_game,home_team_games_without_goals_pre_game,away_team_games_without_goals_pre_game,home_team_cumulative_btts_pre_game,away_team_cumulative_btts_pre_game,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_average_points_per_game_pre_game,away_team_average_points_per_game_pre_game,home_team_wins_in_last_5_games,home_team_draws_in_last_5_games,home_team_losses_in_last_5_games,away_team_wins_in_last_5_games,a

In [35]:
y.value_counts(normalize=True)

result
1    0.436059
2    0.305338
0    0.258604
Name: proportion, dtype: float64

In [36]:
features_scores = mutual_info_classif(X[num_features], y, random_state=42)

In [51]:
pd.DataFrame(dict(zip(num_features, features_scores)), index=['mutual_info_score']).T.sort_values(by='mutual_info_score', ascending=False)

,mutual_info_score
odds_ft_home_team_win,0.093710
odds_ft_away_team_win,0.092769
home_team_shots_on_target,0.090075
away_team_shots_on_target,0.082284
away_team_xg,0.045290
home_team_xg,0.036807
away_team_shots,0.034007
home_team_shots,0.025287
away_team_average_points_per_game_pre_game,0.024887
average_ball_possession_by_away_team_in_last_5_games,0.024528


# Result prediction

## Random forest

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

### Base

In [56]:
rf_model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)

In [57]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=3, random_state=42)

In [58]:
y_pred = rf_model.predict(X_test)

In [59]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.03      0.07      1951
           1       0.53      0.90      0.67      3268
           2       0.57      0.47      0.52      2249

    accuracy                           0.54      7468
   macro avg       0.66      0.47      0.42      7468
weighted avg       0.63      0.54      0.46      7468



### Hyperparameters tuning with optuna

In [168]:
def objective(trial):
    # Number of trees in random forest
    n_estimators = trial.suggest_int(name="n_estimators", low=100, high=500, step=100)

    # Number of features to consider at every split
    max_features = trial.suggest_categorical(name="max_features", choices=['sqrt', 'log2', None]) 

    # Maximum number of levels in tree
    max_depth = trial.suggest_int(name="max_depth", low=10, high=110, step=20)

    # Minimum number of samples required to split a node
    min_samples_split = trial.suggest_int(name="min_samples_split", low=2, high=10, step=2)

    # Minimum number of samples required at each leaf node
    min_samples_leaf = trial.suggest_int(name="min_samples_leaf", low=1, high=4, step=1)
    
    params = {
        "n_estimators": n_estimators,
        "max_features": max_features,
        "max_depth": max_depth,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf
    }
    model = RandomForestClassifier(random_state=42, **params)
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    # cv_score = cross_val_score(model, X_train, y_train, n_jobs=4, cv=5)
    # mean_cv_accuracy = cv_score.mean()
    return acc

In [169]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2024-07-13 16:38:31,007] A new study created in memory with name: no-name-8088fcba-2a49-4459-9989-09039185b482
[W 2024-07-13 16:40:11,421] Trial 0 failed with parameters: {'n_estimators': 400, 'max_features': None, 'max_depth': 110, 'min_samples_split': 6, 'min_samples_leaf': 3} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\magisterka_env\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Łukasz\AppData\Local\Temp\ipykernel_31148\3511437812.py", line 26, in objective
    model.fit(X_train, y_train)
  File "c:\ProgramData\anaconda3\envs\magisterka_env\Lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\envs\magisterka_env\Lib\site-packages\sklearn\ensemble\_forest.py

KeyboardInterrupt: 

## XGBoost

In [60]:
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=3)

In [61]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=3,
              num_parallel_tree=None, ...)

In [62]:
y_pred = xgb_model.predict(X_test)

In [63]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.26      0.33      1951
           1       0.68      0.81      0.74      3268
           2       0.64      0.68      0.66      2249

    accuracy                           0.63      7468
   macro avg       0.59      0.58      0.58      7468
weighted avg       0.60      0.63      0.61      7468



In [150]:
# Ustawienia dla XGBoost
params = {
    'max_depth': 3,  # głębokość drzew
    'eta': 0.1,      # współczynnik uczenia
    'objective': 'multi:softprob',  # funkcja straty dla klasyfikacji wieloklasowej
    'num_class': 3   # liczba klas wynikowych
}

# Przekształcenie danych do formatu DMatrix, który jest bardziej efektywny dla XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Trenowanie modelu
bst = xgb.train(params, dtrain, num_boost_round=100)

# Predykcja
y_pred_prob = bst.predict(dtest)
y_pred = np.argmax(y_pred_prob, axis=1)  # Wybieramy klasę z najwyższym prawdopodobieństwem

# Ocena modelu
accuracy = accuracy_score(y_test, y_pred)
print("Dokładność klasyfikacji XGBoost: {:.2f}%".format(accuracy * 100))

Dokładność klasyfikacji XGBoost: 62.91%


In [64]:
import xgboost as xgb
# from sklearn.metrics import mean_squared_error
import optuna

def objective(trial):
    params = {
        "objective": "multi:softprob",
        # "objective": "multi:softmax",
        # "num_class": "3",
        "n_estimators": 1000,
        "verbosity": 0,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        # "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
    }

    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train, verbose=False)
    
    predictions = model.predict(X_test)
    acc = accuracy_score(y_test, predictions)
    return acc

In [65]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2024-07-14 15:36:48,491] A new study created in memory with name: no-name-ace3165d-7266-4987-8550-de89ec0e86c8
[I 2024-07-14 15:36:59,232] Trial 0 finished with value: 0.5938671665773969 and parameters: {'learning_rate': 0.0031018087274741766, 'max_depth': 2, 'subsample': 0.6917962983186509, 'colsample_bytree': 0.5121727666643779, 'min_child_weight': 1}. Best is trial 0 with value: 0.5938671665773969.
[I 2024-07-14 15:37:42,233] Trial 1 finished with value: 0.5871719335832887 and parameters: {'learning_rate': 0.0016695431307818262, 'max_depth': 8, 'subsample': 0.9054826412985704, 'colsample_bytree': 0.12880535337806984, 'min_child_weight': 7}. Best is trial 0 with value: 0.5938671665773969.
[I 2024-07-14 15:37:53,175] Trial 2 finished with value: 0.6360471344402785 and parameters: {'learning_rate': 0.07039596811280739, 'max_depth': 2, 'subsample': 0.5756458476465504, 'colsample_bytree': 0.3469167984940352, 'min_child_weight': 3}. Best is trial 2 with value: 0.6360471344402785.
[I 20

In [73]:
y_test

27295    2
15565    2
18623    1
26929    1
35578    1
        ..
32348    0
9068     0
8230     1
15878    0
6410     2
Name: result, Length: 7468, dtype: int32

In [67]:
xgb_optuna_best_params = study.best_params

In [75]:
xgb_optuna_best_params['objective'] = 'multi:softprob'

In [79]:
xgb_optuna_best_params['num_class'] = 3

In [80]:
xgb_optuna_best_params

{'learning_rate': 0.018854188545073352,
 'max_depth': 6,
 'subsample': 0.7473497986534416,
 'colsample_bytree': 0.42759609597862747,
 'min_child_weight': 12,
 'objective': 'multi:softprob',
 'num_class': 3}

In [81]:
#converting data to dmatrix format is more effective for xgboost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [82]:
#training xgboost on optimal params from optuna
bst = xgb.train(xgb_optuna_best_params, dtrain, num_boost_round=100)

In [83]:
y_pred_prob

array([0.975096  , 0.87281215, 0.949675  , ..., 0.92944056, 0.8956829 ,
       1.4719692 ], dtype=float32)

In [84]:
#prediction
y_pred_prob = bst.predict(dtest)
y_pred = np.argmax(y_pred_prob, axis=1)

In [86]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.55      0.10      0.17      1951
           1       0.63      0.87      0.73      3268
           2       0.61      0.70      0.65      2249

    accuracy                           0.62      7468
   macro avg       0.60      0.56      0.52      7468
weighted avg       0.60      0.62      0.56      7468



## LightGBM

In [155]:
import lightgbm as lgb

# Ustawienia
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'learning_rate': 0.1,
    'max_depth': 3,
    'num_leaves': 31,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5
}

# Trenowanie
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Model
model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=100)

# Predykcja
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

# Ocena
accuracy = accuracy_score(y_test, y_pred)
print("Dokładność klasyfikacji LightGBM: {:.2f}%".format(accuracy * 100))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019235 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19958
[LightGBM] [Info] Number of data points in the train set: 29871, number of used features: 149
[LightGBM] [Info] Start training from score -1.355019
[LightGBM] [Info] Start training from score -0.830862
[LightGBM] [Info] Start training from score -1.182916
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

### Hyperparameter tuning

In [95]:
def objective(trial):
    param = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'num_class': 3,
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.2),
        'max_depth': trial.suggest_int('max_depth', 1, 10)
    }

    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test)

    model = lgb.train(param, train_data, valid_sets=[test_data])

    y_pred = model.predict(X_test)
    predictions = np.argmax(y_pred, axis=1)
    acc = accuracy_score(y_test, predictions)

    return acc


In [96]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-07-14 16:17:00,259] A new study created in memory with name: no-name-45cab9c6-2255-439f-978f-d561460a9520
C:\Users\Łukasz\AppData\Local\Temp\ipykernel_41160\3038270508.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
C:\Users\Łukasz\AppData\Local\Temp\ipykernel_41160\3038270508.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
C:\Users\Łukasz\AppData\Local\Temp\ipykernel_41160\3038270508.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/

In [99]:
lgbm_optuna_best_params = study.best_params

In [98]:
study.best_value

0.6373861810391002

## Catboost

In [102]:
# Inicjalizacja modelu
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=3,
    loss_function='MultiClass',  # dla klasyfikacji wieloklasowej
    verbose=True
)

# Trenowanie modelu
model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=10)

# Predykcja
y_pred = model.predict(X_test)

# Ocena modelu
accuracy = accuracy_score(y_test, y_pred)
print("Dokładność klasyfikacji CatBoost: {:.2f}%".format(accuracy * 100))

0:	learn: 1.0686533	test: 1.0688561	best: 1.0688561 (0)	total: 40.3ms	remaining: 40.3s
1:	learn: 1.0445600	test: 1.0447754	best: 1.0447754 (1)	total: 76.9ms	remaining: 38.4s
2:	learn: 1.0261194	test: 1.0262857	best: 1.0262857 (2)	total: 114ms	remaining: 38s
3:	learn: 1.0127644	test: 1.0129492	best: 1.0129492 (3)	total: 156ms	remaining: 38.9s
4:	learn: 0.9975257	test: 0.9981369	best: 0.9981369 (4)	total: 210ms	remaining: 41.8s
5:	learn: 0.9852412	test: 0.9860121	best: 0.9860121 (5)	total: 263ms	remaining: 43.5s
6:	learn: 0.9742444	test: 0.9751168	best: 0.9751168 (6)	total: 306ms	remaining: 43.3s
7:	learn: 0.9652384	test: 0.9662150	best: 0.9662150 (7)	total: 337ms	remaining: 41.8s
8:	learn: 0.9568264	test: 0.9576974	best: 0.9576974 (8)	total: 373ms	remaining: 41s
9:	learn: 0.9490877	test: 0.9498515	best: 0.9498515 (9)	total: 404ms	remaining: 40s
10:	learn: 0.9431068	test: 0.9442454	best: 0.9442454 (10)	total: 431ms	remaining: 38.8s
11:	learn: 0.9366286	test: 0.9379061	best: 0.9379061 (11

### Hyperparameter tuning

In [100]:
def objective(trial):
    param = {
        'loss_function': 'MultiClass',
        'iterations': trial.suggest_int('iterations', 50, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
        'border_count': trial.suggest_int('border_count', 5, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_int('random_strength', 0, 100),
        'random_state': 42
    }

    model = CatBoostClassifier(**param)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False, early_stopping_rounds=100)

    y_pred = model.predict(X_test)
    predictions = np.argmax(y_pred, axis=1)
    
    acc = accuracy_score(y_test, predictions)
    
    return acc

In [101]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2024-07-14 16:24:04,305] A new study created in memory with name: no-name-d4b4ff32-4fa9-40c1-bc5b-e62c25399d2e
[I 2024-07-14 16:26:05,393] Trial 0 finished with value: 0.26124799143010174 and parameters: {'iterations': 327, 'learning_rate': 0.10869176717223722, 'depth': 9, 'l2_leaf_reg': 1, 'border_count': 239, 'bagging_temperature': 0.13453406117243294, 'random_strength': 10}. Best is trial 0 with value: 0.26124799143010174.
[W 2024-07-14 16:26:43,242] Trial 1 failed with parameters: {'iterations': 560, 'learning_rate': 0.02101335002607218, 'depth': 10, 'l2_leaf_reg': 10, 'border_count': 19, 'bagging_temperature': 0.980634100238481, 'random_strength': 39} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\magisterka_env\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Łukasz\AppData\Local\Temp\ipykerne

KeyboardInterrupt: 

## Neural Network

In [123]:
np.array(y).reshape(-1, 1)

array([[1],
       [1],
       [2],
       ...,
       [1],
       [2],
       [2]], dtype=int64)

In [124]:
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(np.array(y).reshape(-1, 1))

c:\ProgramData\anaconda3\envs\magisterka_env\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [128]:
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.3, random_state=42)

In [129]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(y_onehot.shape[1], activation='softmax')
])

In [130]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [131]:
model.fit(X_train, y_train, epochs=100, verbose=1)

Epoch 1/100


817/817 [==============================] - 3s 2ms/step - loss: 0.9365 - accuracy: 0.5485
Epoch 2/100
817/817 [==============================] - 1s 1ms/step - loss: 0.8352 - accuracy: 0.6137
Epoch 3/100
817/817 [==============================] - 1s 1ms/step - loss: 0.8115 - accuracy: 0.6231
Epoch 4/100
817/817 [==============================] - 1s 2ms/step - loss: 0.8011 - accuracy: 0.6321
Epoch 5/100
817/817 [==============================] - 1s 1ms/step - loss: 0.7942 - accuracy: 0.6348
Epoch 6/100
817/817 [==============================] - 1s 1ms/step - loss: 0.7885 - accuracy: 0.6396
Epoch 7/100
817/817 [==============================] - 1s 1ms/step - loss: 0.7843 - accuracy: 0.6420
Epoch 8/100
817/817 [==============================] - 1s 2ms/step - loss: 0.7818 - accuracy: 0.6393
Epoch 9/100
817/817 [==============================] - 1s 1ms/step - loss: 0.7781 - accuracy: 0.6445
Epoch 10/100
817/817 [==============================] - 1s 1ms/step - loss: 0.7758 - accu

In [132]:
_, accuracy = model.evaluate(X_test, y_test)

351/351 [==============================] - 1s 2ms/step - loss: 0.8031 - accuracy: 0.6285


In [139]:
X_train

,attendance,game_week,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,home_team_xg,away_team_xg,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_ht_fhg_percentage_pre_match,over_05_ht_fhg_percentage_pre_match,over_15_2hg_percentage_pre_match,over_05_2hg_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,home_team_cumulative_goals_scored_pre_game,away_team_cumulative_goals_scored_pre_game,home_team_cumulative_goals_conceded_pre_game,away_team_cumulative_goals_conceded_pre_game,home_team_average_goals_scored_per_game_pre_game,away_team_average_goals_scored_per_game_pre_game,home_team_average_goals_conceded_per_game_pre_game,away_team_average_goals_conceded_per_game_pre_game,home_team_average_first_half_goals_scored_pre_game,home_team_average_second_half_goals_scored_pre_game,away_team_average_first_half_goals_scored_pre_game,away_team_average_second_half_goals_scored_pre_game,home_team_average_first_half_goals_conceded_pre_game,home_team_average_second_half_goals_conceded_pre_game,away_team_average_first_half_goals_conceded_pre_game,away_team_average_second_half_goals_conceded_pre_game,home_team_average_first_half_goals_total_pre_game,home_team_average_second_half_goals_total_pre_game,away_team_average_first_half_goals_total_pre_game,away_team_average_second_half_goals_total_pre_game,home_team_average_corners_total_per_game_pre_game,away_team_average_corners_total_per_game_pre_game,home_team_average_corners_per_game_pre_game,away_team_average_corners_per_game_pre_game,home_team_average_yellow_cards_total_per_game_pre_game,away_team_average_yellow_cards_total_per_game_pre_game,home_team_average_yellow_cards_per_game_pre_game,away_team_average_yellow_cards_per_game_pre_game,home_team_cumulative_red_cards_pre_game,away_team_cumulative_red_cards_pre_game,home_team_average_red_cards_total_per_game_pre_game,away_team_average_red_cards_total_per_game_pre_game,home_team_average_red_cards_per_game_pre_game,away_team_average_red_cards_per_game_pre_game,home_team_average_shots_per_game_pre_game,home_team_average_shots_on_target_per_game_pre_game,away_team_average_shots_per_game_pre_game,away_team_average_shots_on_target_per_game_pre_game,home_team_average_fouls_per_game_pre_game,away_team_average_fouls_per_game_pre_game,home_team_average_fouls_total_per_game_pre_game,away_team_average_fouls_total_per_game_pre_game,home_team_average_possession_per_game_pre_game,away_team_average_possession_per_game_pre_game,home_team_average_xg_per_game_pre_game,away_team_average_xg_per_game_pre_game,home_team_games_without_goals_pre_game,away_team_games_without_goals_pre_game,home_team_cumulative_btts_pre_game,away_team_cumulative_btts_pre_game,home_team_cumulative_clean_sheets_pre_game,away_team_cumulative_clean_sheets_pre_game,home_team_cumulative_wins_pre_game,home_team_cumulative_draws_pre_game,home_team_cumulative_losses_pre_game,away_team_cumulative_wins_pre_game,away_team_cumulative_draws_pre_game,away_team_cumulative_losses_pre_game,home_team_average_points_per_game_pre_game,away_team_average_points_per_game_pre_game,home_team_wins_in_last_5_games,home_team_draws_in_last_5_games,home_team_losses_in_last_5_games,away_team_wins_in_last_5_games,a

In [146]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [147]:
import numpy as np
import pandas as pd

# Ustawienia początkowe
n_features = 149  # liczba cech
sequence_length = 10  # długość sekwencji

# Funkcja do przekształcania danych wejściowych w sekwencje
def create_sequences(X, y, sequence_length):
    n_samples = len(X) - sequence_length + 1
    X_seq = np.zeros((n_samples, sequence_length, X.shape[1]))
    y_seq = np.zeros(n_samples)

    for i in range(n_samples):
        X_seq[i] = X.iloc[i:i+sequence_length].values
        y_seq[i] = y.iloc[i+sequence_length-1]  # przypisujemy etykietę z ostatniego kroku w sekwencji

    return X_seq, y_seq

# Przetwarzanie X_train i y_train
X_train_seq, y_train_seq = create_sequences(X_train, y_train, sequence_length)

# Przetwarzanie X_test i y_test (zakładam, że y_test również jest dostępny)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, sequence_length)

In [148]:
import tensorflow as tf

# Definiowanie modelu
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, input_shape=(sequence_length, n_features), return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Zakładam klasyfikację binarną; dostosuj w razie potrzeby
])

# Kompilacja modelu
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Dostosuj loss i activation w zależności od problemu

# Trenowanie modelu
history = model.fit(X_train_seq, y_train_seq, epochs=30, validation_data=(X_test_seq, y_test_seq), verbose=1)

Epoch 1/30
934/934 [==============================] - 16s 11ms/step - loss: -1.4234 - accuracy: 0.4354 - val_loss: -2.9558 - val_accuracy: 0.4373
Epoch 2/30
934/934 [==============================] - 9s 9ms/step - loss: -5.6096 - accuracy: 0.4357 - val_loss: -7.7222 - val_accuracy: 0.4373
Epoch 3/30
934/934 [==============================] - 9s 9ms/step - loss: -10.7457 - accuracy: 0.4357 - val_loss: -12.5743 - val_accuracy: 0.4373
Epoch 4/30
934/934 [==============================] - 9s 10ms/step - loss: -16.2755 - accuracy: 0.4357 - val_loss: -17.2603 - val_accuracy: 0.4373
Epoch 5/30
235/934 [======>.......................] - ETA: 8s - loss: -20.2336 - accuracy: 0.4266

KeyboardInterrupt: 

# Score prediction (home_team_goal_count, away_team_goal_count)